In [1]:
import random
import csv

def initialize_population(population_size, num_tasks):
    population = []
    for _ in range(population_size):
        chromosome = random.sample(range(num_tasks), num_tasks)
        population.append(chromosome)
    return population

def calculate_fitness(chromosome, tasks, machines, in_times):
    completion_times = [0] * machines
    schedule = [[] for _ in range(machines)]  # Store the schedule for each machine

    for i, task_id in enumerate(chromosome):
        machine_id = completion_times.index(min(completion_times))
        start_time = max(completion_times[machine_id], in_times[task_id])
        completion_times[machine_id] = start_time + tasks[task_id]

        # Update the schedule for the machine
        schedule[machine_id].append((task_id, start_time, completion_times[machine_id]))

    makespan = max(completion_times)
    return makespan, schedule

def tournament_selection(population, tasks, machines, in_times, tournament_size):
    tournament = random.sample(population, tournament_size)
    tournament_fitness = [calculate_fitness(chromosome, tasks, machines, in_times) for chromosome in tournament]
    selected_index = tournament_fitness.index(min(tournament_fitness))
    return tournament[selected_index]

def order_crossover(parent1, parent2):
    point1 = random.randint(0, len(parent1) - 1)
    point2 = random.randint(point1, len(parent1) - 1)

    child1 = [-1] * len(parent1)
    child2 = [-1] * len(parent1)

    child1[point1:point2 + 1] = parent1[point1:point2 + 1]
    child2[point1:point2 + 1] = parent2[point1:point2 + 1]

    index = point2 + 1
    while -1 in child1:
        gene = parent2[index % len(parent2)]
        if gene not in child1:
            child1[index % len(parent1)] = gene
        index = (index + 1) % len(parent1)

    index = point2 + 1
    while -1 in child2:
        gene = parent1[index % len(parent1)]
        if gene not in child2:
            child2[index % len(parent1)] = gene
        index = (index + 1) % len(parent1)

    return child1, child2

def mutate(chromosome):
    point1 = random.randint(0, len(chromosome) - 1)
    point2 = random.randint(0, len(chromosome) - 1)
    while point1 == point2:
        point2 = random.randint(0, len(chromosome) - 1)
    chromosome[point1], chromosome[point2] = chromosome[point2], chromosome[point1]

def read_data_from_csv(filename):
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        data = [list(map(int, row)) for row in reader]

    num_machines = data[0][0]
    num_tasks = data[0][1]
    task_data = data[1:1 + num_tasks]

    in_times = [row[0] for row in task_data]
    burst_times = [row[1] for row in task_data]

    return num_machines, num_tasks, in_times, burst_times

def print_schedule(schedule):
    for machine_id, tasks in enumerate(schedule):
        print(f"Machine {machine_id + 1} Schedule:")
        for task_id, start_time, end_time in tasks:
            print(f"Task {task_id + 1}: Start Time = {start_time}, End Time = {end_time}")

def genetic_algorithm(num_tasks, num_machines, in_times, burst_times, population_size, generations, tournament_size, crossover_rate, mutation_rate):
    tasks = burst_times
    machines = num_machines
    population = initialize_population(population_size, num_tasks)

    for generation in range(generations):
        parents = [tournament_selection(population, tasks, machines, in_times, tournament_size) for _ in range(population_size)]

        offspring = []

        for i in range(0, population_size, 2):
            if random.random() < crossover_rate:
                child1, child2 = order_crossover(parents[i], parents[i + 1])
                offspring.extend([child1, child2])
            else:
                offspring.extend([parents[i], parents[i + 1]])

        population = offspring.copy()

        for i in range(population_size):
            if random.random() < mutation_rate:
                mutate(population[i])

        # Print the schedule for the best chromosome in each generation
        best_chromosome = min(population, key=lambda x: calculate_fitness(x, tasks, machines, in_times)[0])
        best_makespan, best_schedule = calculate_fitness(best_chromosome, tasks, machines, in_times)
        print(f"Generation {generation + 1} - Best Makespan: {best_makespan}")
        print_schedule(best_schedule)

    # Return the best solution found
    best_chromosome = min(population, key=lambda x: calculate_fitness(x, tasks, machines, in_times)[0])
    best_makespan, best_schedule = calculate_fitness(best_chromosome, tasks, machines, in_times)
    return best_chromosome, best_makespan

if __name__ == "__main__":
    filename = 'generated_data_1.csv'
    num_machines, num_tasks, in_times, burst_times = read_data_from_csv(filename)

    population_size = 50
    generations = 100
    tournament_size = 5
    crossover_rate = 0.8
    mutation_rate = 0.2

    best_chromosome, best_makespan = genetic_algorithm(num_tasks, num_machines, in_times, burst_times, population_size, generations, tournament_size, crossover_rate, mutation_rate)
    # print("\nFinal Best Chromosome:", best_chromosome)
    print("Final Best Makespan:", best_makespan)


Generation 1 - Best Makespan: 9
Machine 1 Schedule:
Task 3: Start Time = 6, End Time = 9
Machine 2 Schedule:
Task 4: Start Time = 2, End Time = 6
Task 1: Start Time = 6, End Time = 7
Task 2: Start Time = 7, End Time = 9
Generation 2 - Best Makespan: 9
Machine 1 Schedule:
Task 1: Start Time = 2, End Time = 3
Task 2: Start Time = 4, End Time = 6
Task 3: Start Time = 6, End Time = 9
Machine 2 Schedule:
Task 4: Start Time = 2, End Time = 6
Generation 3 - Best Makespan: 9
Machine 1 Schedule:
Task 1: Start Time = 2, End Time = 3
Task 2: Start Time = 4, End Time = 6
Task 3: Start Time = 6, End Time = 9
Machine 2 Schedule:
Task 4: Start Time = 2, End Time = 6
Generation 4 - Best Makespan: 9
Machine 1 Schedule:
Task 1: Start Time = 2, End Time = 3
Task 2: Start Time = 4, End Time = 6
Task 3: Start Time = 6, End Time = 9
Machine 2 Schedule:
Task 4: Start Time = 2, End Time = 6
Generation 5 - Best Makespan: 9
Machine 1 Schedule:
Task 1: Start Time = 2, End Time = 3
Task 2: Start Time = 4, End Tim